In [3]:
##pandas时间数据处理
import pandas as pd

date_strings = ['2025-10-09', '10/10/2025', 'Oct 11, 2025', '2025.10.12']

# 直接使用 format='mixed' 参数
# 这会告诉 Pandas 放弃寻找统一格式，直接进入逐个解析的模式
time_series = pd.to_datetime(date_strings, format='mixed')

print(time_series)

##相同效果，因为date_string中含有多种时间数据类型，因此报错了
import pandas as pd

date_strings = ['2025-10-09', '10/10/2025', 'Oct 11, 2025', '2025.10.12']

# 对列表中的每一个日期字符串单独调用 pd.to_datetime
# 然后将结果重新组合成一个 DatetimeIndex
time_series = pd.to_datetime([pd.to_datetime(d) for d in date_strings])

print(time_series)

DatetimeIndex(['2025-10-09', '2025-10-10', '2025-10-11', '2025-10-12'], dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2025-10-09', '2025-10-10', '2025-10-11', '2025-10-12'], dtype='datetime64[ns]', freq=None)


In [4]:
#format的使用：
import pandas as pd

dates_str = ['09/10/2025', '15/10/2025', '20/11/2025']

# 直接转换可能会被误解为 "月/日/年"
# pd.to_datetime(dates_str) -> [2025-09-10, 2025-10-15, ...] 错误！

# 使用 format 参数明确指定格式
# %d 匹配 '09', '/' 匹配 '/', %m 匹配 '10', %Y 匹配 '2025'
date_times = pd.to_datetime(dates_str, format='%d/%m/%Y')

print(date_times)

DatetimeIndex(['2025-10-09', '2025-10-15', '2025-11-20'], dtype='datetime64[ns]', freq=None)


In [22]:
##将时间设为索引：要解锁 Pandas 的时间序列分析能力，最关键的一步就是将时间列设置成 DataFrame 的索引
# 创建一个示例 DataFrame
df = pd.DataFrame({
    'date_str': ['2025-10-01', '2025-10-02', '2025-10-03', '2025-10-04'],
    'sales': [200, 220, 250, 230]
})

# 1. 将字符串列转换为 datetime 类型
df['date'] = pd.to_datetime(df['date_str'])

# 2. 将 'date' 列设为索引
df = df.set_index('date')

# 3. (可选) 删除原来的字符串列
df = df.drop('date_str', axis=1)

print(df)
print("\n索引类型:", type(df.index))

            sales
date             
2025-10-01    200
2025-10-02    220
2025-10-03    250
2025-10-04    230

索引类型: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [23]:
##pd.date_range()：生成时间序列
# 创建从2025年10月1日开始，连续7天的日期
date_index = pd.date_range(start='2025-10-01', periods=7, freq='D')
print(date_index)

DatetimeIndex(['2025-10-01', '2025-10-02', '2025-10-03', '2025-10-04',
               '2025-10-05', '2025-10-06', '2025-10-07'],
              dtype='datetime64[ns]', freq='D')


In [25]:
# 使用上面的 df
# 筛选特定某一天
print(df.loc['2025-10-02'])

# 筛选某一年
df_2024 = df.loc['2025']
print(df_2025)

# 筛选某一月
df_oct = df.loc['2025-10']
print("\n筛选2025年10月的数据:\n", df_oct)

# 筛选一个日期范围（切片）
date_range_slice = df.loc['2025-10-01':'2025-10-03']
print("\n筛选日期范围:\n", date_range_slice)

sales    220
Name: 2025-10-02 00:00:00, dtype: int64
            sales
date             
2025-10-01    200
2025-10-02    220
2025-10-03    250
2025-10-04    230

筛选2025年10月的数据:
             sales
date             
2025-10-01    200
2025-10-02    220
2025-10-03    250
2025-10-04    230

筛选日期范围:
             sales
date             
2025-10-01    200
2025-10-02    220
2025-10-03    250


In [20]:
##重采样 (Resampling)：改变时间频率
#这是时间序列分析的核心操作之一，比如将日度数据聚合为月度数据，或将月度数据扩展为日度数据。

#a) 降采样 (Downsampling)：高频 -> 低频
idx = pd.date_range(start='2025-09-25', end='2025-10-05', freq='D')
sales_data = pd.Series(range(len(idx)), index=idx) * 10

# 'ME' 代表按月（Month End）重采样，并对每个月的数据求和
monthly_sales = sales_data.resample('ME').sum()
print("\n每月销售总额:\n", monthly_sales)

# 'W' 代表按周（Week End, 默认为周日）重采样，并计算每周的平均值
weekly_avg_sales = sales_data.resample('W').mean()
print("\n每周平均销售额:\n", weekly_avg_sales)


每月销售总额:
 2025-09-30    150
2025-10-31    400
Freq: ME, dtype: int64

每周平均销售额:
 2025-09-28    15.0
2025-10-05    70.0
Freq: W-SUN, dtype: float64


In [33]:
#b）升采样 (Upsampling)：低频 -> 高频
#例如，将月度数据扩展为日度数据。这会产生很多缺失值（NaN），需要填充方法（如 ffill 向前填充, bfill 向后填充）。
# 假设我们只有月度数据
monthly_data = pd.Series([100, 120], index=pd.to_datetime(['2025-01-31', '2025-02-28']))

# 'D' 代表按天重采样，并用前一个有效值填充
daily_data = monthly_data.resample('D').ffill()
print("\n月度数据扩展为日度数据:\n", daily_data)


月度数据扩展为日度数据:
 2025-01-31    100
2025-02-01    100
2025-02-02    100
2025-02-03    100
2025-02-04    100
2025-02-05    100
2025-02-06    100
2025-02-07    100
2025-02-08    100
2025-02-09    100
2025-02-10    100
2025-02-11    100
2025-02-12    100
2025-02-13    100
2025-02-14    100
2025-02-15    100
2025-02-16    100
2025-02-17    100
2025-02-18    100
2025-02-19    100
2025-02-20    100
2025-02-21    100
2025-02-22    100
2025-02-23    100
2025-02-24    100
2025-02-25    100
2025-02-26    100
2025-02-27    100
2025-02-28    120
Freq: D, dtype: int64


In [ ]:
#常用频率代码：D(天), W(周), M(月), Q(季), A或Y(年)。

In [34]:

## 4. 时间特征提取 (Feature Engineering)
#从 DatetimeIndex 中可以轻松提取出年、月、日、星期几等信息，这些对于机器学习模型非常有用。这通过 .dt 访问器实现。
# 使用之前的 df
# 提取星期几 (Monday=0, Sunday=6)
df['day_of_week'] = df.index.dayofweek

# 提取星期名称
df['day_name'] = df.index.day_name()

# 提取是否是月份的开始
df['is_month_start'] = df.index.is_month_start

print("\n提取时间特征:\n", df)


提取时间特征:
             sales  day_of_week   day_name  is_month_start
date                                                     
2025-10-01    200            2  Wednesday            True
2025-10-02    220            3   Thursday           False
2025-10-03    250            4     Friday           False
2025-10-04    230            5   Saturday           False


In [35]:
# 5. 窗口函数 (Window Functions)
#用于计算移动平均、移动总和等。

# a) 移动窗口 (Rolling Windows)
# 一个固定大小的窗口，在数据上滑动，对窗口内的数据进行计算。

# 计算3日移动平均销售额
df['3D_rolling_avg'] = df['sales'].rolling(window=3).mean()
print("\n计算3日移动平均:\n", df)


计算3日移动平均:
             sales  day_of_week   day_name  is_month_start  3D_rolling_avg
date                                                                     
2025-10-01    200            2  Wednesday            True             NaN
2025-10-02    220            3   Thursday           False             NaN
2025-10-03    250            4     Friday           False      223.333333
2025-10-04    230            5   Saturday           False      233.333333


In [ ]:
#b)扩张窗口 (Expanding Windows)
#窗口从头开始，不断扩大，包含至今为止的所有数据，用于计算累计值。
# 计算累计销售总额
df['cumulative_sum'] = df['sales'].expanding().sum()
print("\n计算累计总和:\n", df)